In [23]:
import numpy as np
from scipy.special import jv
from numba import jit,autojit

In [18]:
def dbesj(x, alpha, n):
    """sequence of bessel functions where n is the number of elements of a 
    bessel function of J_(alpha + k -1)/x where k = 1...n
    """
    bessel_j = np.zeros(n+1)
    i = 0
    for k in range(1,n+1):
       bessel_j[i] = jv(alpha + k -1, x)
       i = i + 1
    y = sum([num for num in bessel_j if num== 0])
    return bessel_j, y

In [19]:
x = 500
alpha = 0

In [20]:
%timeit dbesj(x,alpha,100)

1000 loops, best of 3: 518 µs per loop


In [24]:
@autojit
def find_norm(z):
    """find complex norm^2 of a vector of complex numbers"""
    return sum([i.real**2 + i.imag**2 for i in z])

In [4]:
z = np.zeros(10, dtype = complex)

In [14]:
z
for i in range(len(z)):
    z[i] = np.complex(np.random.rand(),np.random.rand())

In [15]:
z

array([ 0.97618597+0.82296229j,  0.98270758+0.78309954j,
        0.05156616+0.26593888j,  0.48047802+0.89968593j,
        0.30941247+0.4860372j ,  0.43273665+0.9055892j ,
        0.25794902+0.11454549j,  0.64422296+0.76807301j,
        0.28462320+0.54922502j,  0.81324881+0.33384545j])

In [25]:
%timeit find_norm(z)

NotImplementedError: offset=9 opcode=0x84 opname=MAKE_FUNCTION